# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Обзор данных

Для начала выведем на экран первые 10 строк датафрейма и изучим его.

In [ ]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в df
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Также посмотрим общую информацию о таблице. 

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Мы видим 11 столбов, их типы, а также, что в 2 столбцах есть пропущенные данные. 

Выведем на экран все названия столбцов, чтобы проверить правильность их написания.

In [ ]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Все названия столбцов правильного формата.

## Шаг 2.1 Заполнение пропусков

Проверяем количество пустых ячеек в таблице

In [ ]:
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения есть в столбце 'days_employed' и 'total_income'

Для начала еще раз посмотрим несколько строк таблицы и обратим внимание на 'days_employed' и 'total_income'

In [ ]:
display(df.head(15))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Столбец 'days_employed' состоит из отрицательных и положительных значений. Узнаем сколько отрицательных и положительных значений в столбце, а  также другие аналитические данные.

In [ ]:
print('Отрицательные значения = ', df[df['days_employed']<0]['days_employed'].count())
print('Положительные значения = ', df[df['days_employed']>0]['days_employed'].count())
print()
print(df['days_employed'].describe())

Отрицательные значения =  15906
Положительные значения =  3445

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64


Отрицательные значения в столбце, возможно появились из-за того, что при вводе данных использовали тире. Значения лучше всего перевести из дней трудового стажа в года.

In [ ]:
df['days_employed'] = abs(df['days_employed'])/365
display(df.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В столбце 'days_employed' есть аномальные значения, например в строчке №4 стаж работы 932 года. Посмотрим, как много таких показателей. Максимальный стаж работы не может превышать 70 лет. Выведем количество значений со стажем работы более 50 лет.

In [ ]:
df[df['days_employed'] > 50]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,1096.660649,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,927.539597,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,996.023258,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,919.401832,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21505,0,928.506483,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439.993167,сыграть свадьбу
21508,0,1058.897847,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638.590915,недвижимость
21509,0,992.222066,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029.059379,операции с недвижимостью
21518,0,1024.645783,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем


Аномальных значений больше 3400, это довольно много. Лучше всего у коллег уточнить причину аномалии данных. Но т.к. у нас нет такой возможности, то продолжаем работать дальше.

In [ ]:
df.groupby('income_type')['days_employed'].count()

income_type
безработный            2
в декрете              1
госслужащий         1312
компаньон           4577
пенсионер           3443
предприниматель        1
сотрудник          10014
студент                1
Name: days_employed, dtype: int64

Аномальные значения появляются в основном у пенсионеров и безработных. Как вариант, мы можем заменить аномальные значения средними значениями стажа работы исходя из возраста человека. Но мы рассмотрим гипотезу, что в этой группе стаж работы записан не в днях, а в часах. Для этого данные в группе разделим на 24.

In [ ]:
df['days_employed'].where(~(df.days_employed > 100), other=df['days_employed'] / 24, inplace=True)
df[df['days_employed'] > 100]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Данные каждутся более реалистичными. Проверим есть ли пропуски в столбце со стажем работы.

In [ ]:
display(df.loc[df['days_employed'].isna()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Заполним все пропущенные значения медианой возрастной группы. Для этого создадим несколько категорий.

In [ ]:
def years_category(row):
    if row['dob_years'] < 26:
        return 'youth'
    if row['dob_years'] < 37:
        return 'middle'
    if row['dob_years'] < 48:
        return 'adult'
    if row['dob_years'] < 61:
        return 'adult_pluse'
    return 'old'

df['years_category'] = df.apply(years_category, axis=1)
print(df['years_category'].value_counts())

adult_pluse    6102
adult          6033
middle         5915
old            2140
youth          1335
Name: years_category, dtype: int64


In [ ]:
df.groupby('years_category')['days_employed'].median()

years_category
adult           5.370677
adult_pluse    11.518305
middle          3.800916
old            40.697567
youth           2.318393
Name: days_employed, dtype: float64

Заменяем пропущенные значения медианными.

In [ ]:
vallues = df.groupby('years_category')['days_employed'].transform('median')
df['days_employed'] = df['days_employed'].fillna(value=vallues)
#Для проверки выведем строки где были заменены пропуски
display(df.loc[[12,26,29, 41, 55, 65,72, 82]])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category
12,0,40.697567,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,old
26,0,5.370677,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,adult
29,0,40.697567,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,old
41,0,11.518305,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,adult_pluse
55,0,11.518305,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,adult_pluse
65,0,2.318393,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью,youth
72,1,3.800916,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью,middle
82,2,11.518305,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье,adult_pluse


Cтолбец 'total_income' важен для анализа данных, поэтому все пропуски заполняем медианными значениями по категориям 'income_type'. Для начала выведем на экран несколько строк с пропущенными значениями в 'total_income'.

In [ ]:
display(df.loc[df['total_income'].isna()].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category
12,0,40.697567,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,old
26,0,5.370677,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,adult
29,0,40.697567,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,old
41,0,11.518305,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,adult_pluse
55,0,11.518305,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,adult_pluse
65,0,2.318393,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью,youth
67,0,11.518305,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи,adult_pluse
72,1,3.800916,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью,middle
82,2,11.518305,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье,adult_pluse
83,0,11.518305,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье,adult_pluse


Заполним пропущенные данные медианными значениями по категориям 'income_type'

In [ ]:
df.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

In [ ]:
vallues_income = df.groupby('income_type')['total_income'].transform('median')
df['total_income'] = df['total_income'].fillna(value=vallues_income)
#Для проверки выведем строки где были заменены пропуски
display(df.loc[[12,26,29, 41, 55, 65,72, 82]])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category
12,0,40.697567,65,среднее,1,гражданский брак,1,M,пенсионер,0,118514.486412,сыграть свадьбу,old
26,0,5.370677,41,среднее,1,женат / замужем,0,M,госслужащий,0,150447.935283,образование,adult
29,0,40.697567,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,118514.486412,строительство жилой недвижимости,old
41,0,11.518305,50,среднее,1,женат / замужем,0,F,госслужащий,0,150447.935283,сделка с подержанным автомобилем,adult_pluse
55,0,11.518305,54,среднее,1,гражданский брак,1,F,пенсионер,1,118514.486412,сыграть свадьбу,adult_pluse
65,0,2.318393,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,172357.950966,операции с коммерческой недвижимостью,youth
72,1,3.800916,32,высшее,0,женат / замужем,0,M,госслужащий,0,150447.935283,операции с коммерческой недвижимостью,middle
82,2,11.518305,50,высшее,0,женат / замужем,0,F,сотрудник,0,142594.396847,жилье,adult_pluse


Заменим вещественный тип данных в столбце total_income на целочисленный.

In [ ]:
df['total_income'] = df['total_income'].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
 12  years_category    21525 non-null  object 
dtypes: float64(1), int64(6), object(6)
memory usage: 2.1+ MB


# Шаг 2.2 Проверка данных на аномалии и исправления. Удаление дубликатов.

Проверим дубликаты в строчках

In [ ]:
df.duplicated().sum()

54

Удаляем дубликаты

In [ ]:
df.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,adult
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,middle
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,middle
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,middle
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,adult_pluse
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,12.409087,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,adult
21467,0,39.262261,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,old
21468,1,5.789991,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,adult
21469,3,8.527347,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,adult


Поочередно проверим все столбцы на аномалии и при необходимости исправим их, а также избавимся от дублей.

### Столбец 'children'

In [ ]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Подозрительным кажутся 75 строк с 20 детьми и 47 строк с -1. Т.к. строк не много и мы не можем обратиться к коллегам за уточнением данных, то удалим строчки.

In [ ]:
df = df[df['children'] != 20]
df = df[df['children'] != -1]
df['children'].value_counts()

0    14149
1     4818
2     2055
3      330
4       41
5        9
Name: children, dtype: int64

### Столбец 'dob_years'

In [ ]:
df['dob_years'].value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
48    536
30    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
53    457
57    457
51    446
59    441
55    441
26    406
60    376
25    356
61    353
62    351
63    268
24    263
64    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Возраст 0 явлеяется ошибкой, таких ячеек 100. Таким небольшим количеством данных можно пренебреч. Удалим их и проверим результа.

In [ ]:
df = df[df['dob_years'] != 0]
df[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category


### Столбец 'education' и 'education_id'

In [ ]:
df['education'].value_counts()

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

Исбавимся от дублей и еще раз посчитаем посмотрим на данные. 

In [ ]:
df['education'] = df['education'].str.lower()
df['education'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

In [ ]:
df['education_id'].value_counts()

1    15073
0     5202
2      739
3      282
4        6
Name: education_id, dtype: int64

### столбец 'family_status' и 'family_status_id'

In [ ]:
print(df['family_status'].value_counts())
print(df['family_status_id'].value_counts())

женат / замужем          12254
гражданский брак          4139
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64
0    12254
1     4139
4     2783
3     1179
2      947
Name: family_status_id, dtype: int64


### стоблец 'gender'

In [ ]:
df['gender'].value_counts()

F      14083
M       7218
XNA        1
Name: gender, dtype: int64

Одна ячейка с неизвестным полом. Удалим ее.

In [ ]:
df = df[df['gender'] != 'XNA']
df['gender'].value_counts()

F    14083
M     7218
Name: gender, dtype: int64

### столбец 'debt'

In [ ]:
df['debt'].value_counts()

0    19577
1     1724
Name: debt, dtype: int64

### столбец 'purpose'

In [ ]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           764
операции с недвижимостью                  670
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с коммерческой недвижимостью     644
операции с жильем                         642
покупка жилья для семьи                   639
жилье                                     636
покупка жилья                             635
недвижимость                              628
операции со своей недвижимостью           626
строительство собственной недвижимости    626
строительство недвижимости                620
строительство жилой недвижимости          619
покупка своего жилья                      618
покупка недвижимости                      615
ремонт жилью                              607
покупка жилой недвижимости                600
на покупку своего автомобиля              501
заняться высшим образованием      

Многие цели кредита похожи. По сути это скрытые дубли. В процессе категоризация (в разделе ниже) сведем все цели к небольшому количеству групп.

## Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [ ]:
educ_log = df[['education_id', 'education']]
display(educ_log)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
...,...,...
21520,1,среднее
21521,1,среднее
21522,1,среднее
21523,1,среднее


Удаляем из исходного датафрейма столбец 'education'

In [ ]:
df.drop(labels='education', axis=1)

,children,days_employed,dob_years,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category
0,1,23.116912,42,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,adult
1,1,11.026860,36,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,middle
2,0,15.406637,33,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,middle
3,3,11.300677,32,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,middle
4,0,38.843159,53,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,adult_pluse
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,12.409087,43,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,adult
21521,0,39.262261,67,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,old
21522,1,5.789991,38,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,adult
21523,3,8.527347,38,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,adult


In [ ]:
family_stat_log = df[['family_status_id', 'family_status']]
display(family_stat_log)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
...,...,...
21520,1,гражданский брак
21521,0,женат / замужем
21522,1,гражданский брак
21523,0,женат / замужем


Удаляем из исходного датафрейма столбец 'family_status'

In [ ]:
df.drop(labels='family_status', axis=1)

,children,days_employed,dob_years,education,education_id,family_status_id,gender,income_type,debt,total_income,purpose,years_category
0,1,23.116912,42,высшее,0,0,F,сотрудник,0,253875,покупка жилья,adult
1,1,11.026860,36,среднее,1,0,F,сотрудник,0,112080,приобретение автомобиля,middle
2,0,15.406637,33,среднее,1,0,M,сотрудник,0,145885,покупка жилья,middle
3,3,11.300677,32,среднее,1,0,M,сотрудник,0,267628,дополнительное образование,middle
4,0,38.843159,53,среднее,1,1,F,пенсионер,0,158616,сыграть свадьбу,adult_pluse
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,12.409087,43,среднее,1,1,F,компаньон,0,224791,операции с жильем,adult
21521,0,39.262261,67,среднее,1,0,F,пенсионер,0,155999,сделка с автомобилем,old
21522,1,5.789991,38,среднее,1,1,M,сотрудник,1,89672,недвижимость,adult
21523,3,8.527347,38,среднее,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,adult


## Шаг 2.6. Категоризация дохода.

In [ ]:
def income_category(row):
    if row['total_income'] < 30001:
        return 'E'
    if 30000 < row['total_income'] < 50001:
        return 'D'
    if 50000 < row['total_income'] < 200001:
        return 'C'
    if 200000 < row['total_income'] < 1000001:
        return 'B'
    if row['total_income'] > 1000001:
        return 'A'

df['total_income_category'] = df.apply(income_category, axis=1)
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category,total_income_category
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,adult,B
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,middle,C
2,0,15.406637,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,middle,C
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,middle,B
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,adult_pluse,C


## Шаг 2.7. Категоризация целей кредита.

In [ ]:
def purpose_grouping(row):
    if 'авто' in row['purpose'] :
        return 'операции с автомобилем'
    if 'недвижимост' in row['purpose']  or 'жиль' in row['purpose']:
        return 'операции с недвижимостью'
    if 'свадьб' in row['purpose']:
        return 'проведение свадьбы'
    if 'образова' in row['purpose']:
        return 'получение образования'

df['purpose_category'] = df.apply(purpose_grouping, axis=1)
display(df.head())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_category,total_income_category,purpose_category
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,adult,B,операции с недвижимостью
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,middle,C,операции с автомобилем
2,0,15.406637,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,middle,C,операции с недвижимостью
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,middle,B,получение образования
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,adult_pluse,C,проведение свадьбы


## Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

Для эффективного анализа данных и ответа на наши вопросы лучше всего подойдет создание сводной таблицы.

In [ ]:
df.pivot_table(index='children', values='debt')

,debt
children,
0,0.075147
1,0.091837
2,0.095005
3,0.082317
4,0.097561
5,0.000000


У клиентов, имеющих 5 детей 100% возврат кредита. Стоит проверить какой объем выборки в этом сегменте.

In [ ]:
df['children'].value_counts()

0    14079
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

На мой взгляд выбрка клиентов с 5 детьми слишком маленькая, чтобы сделать реальные выводы. Тоже касается и клиентов с 4 детьми. Поэтому я бы их убрала из анализа.

##### Вывод 1:
Нет четкой зависимости между количеством детей и возвратом кредита. Но прослеживается небольшая зависимость в том, что клиенты без детей чаще возвращают кредит, чем клиенты с детьми. Количество детей на возврат кредита сильно не влияет.

##### Вопрос 2:  Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
display(df.pivot_table(index='family_status', values='debt'))
df['family_status'].value_counts()

,debt
family_status,
Не женат / не замужем,0.097736
в разводе,0.071247
вдовец / вдова,0.065470
гражданский брак,0.092557
женат / замужем,0.075322


женат / замужем          12254
гражданский брак          4138
Не женат / не замужем     2783
в разводе                 1179
вдовец / вдова             947
Name: family_status, dtype: int64

##### Вывод 2:
Клиенты, которые состоят в официальном или гражданском браке намного реже имеют задолженности по кредиту, чем люди вне брака или вдовцы. Возможно, это можно объяснить двойным доходом семьи.

##### Вопрос 3:  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
display(df.pivot_table(index='total_income_category', values='debt'))
df['total_income_category'].value_counts()

,debt
total_income_category,
A,0.080000
B,0.070784
C,0.084548
D,0.060519
E,0.090909


C    15920
B     4987
D      347
A       25
E       22
Name: total_income_category, dtype: int64

##### Вывод 3:
Мы видим, что в категории A и E слишком маленькие выборки, поэтому делать выводы по этим категориям не стоит. Изучив, сводную таблицу, можно сделать вывод, что есть разница в доле возврата кредита в различных категориях, но нет зависимости от роста дохода в целом. 

##### Вопрос 4:  Как разные цели кредита влияют на его возврат в срок?

In [ ]:
df.pivot_table(index='purpose_category', values='debt')

,debt
purpose_category,
операции с автомобилем,0.093040
операции с недвижимостью,0.072400
получение образования,0.092737
проведение свадьбы,0.077916


##### Вывод 4:
Доля возврата кредита на недвижимость и свадьбу выше, чем на образование и автомобиль. 

## Общий вывод:

При предобработке данных было удалено около 1% некорректных строк и 1 некорректный столбец(который для поставленных задач не был необходим). Поменяли в двух столбцах типы - для лучшей наглядности и более быстрой обработки. Удалили дубликаты, заполнили пропуски. Для двух столбцом сделали категоризацию. После предобработки, проанализировали данные с помощью сводных таблиц и ответили на все поставленные вопросы. Были выявили закономерности в просрочках по кредитам(где-то ярко выраженные, а где-то нет).

К клиентам с наибольшей вероятностью просрочки относятся люди не состоящие в официальном браке, имеющие 1-2 детей и с целью кредита на автомобиль или образование. Уровень дохода существенно на долю возвратов кредитов не влияет.
Наилучшим клиентом являются люди без детей, состоящие в официальном браке и с целью кредита на недвижимость или свадьбу.